In [1]:
import pandas as pd
import csv
import re
from Bio import SeqIO
import os

# Data Processing

Read in and format the raw data from running CLEAN-Contact, CLEAN, and from grabbing data from BioCyc on Prochlorococcus marinus subsp. pastoris str. CCMP1986, referred to here as MED4. These changes will make them easier to compare and analyze later.


In [2]:
# relative paths for any OS
# clean-contact results
cc_path = os.path.join('..', 'raw-data', 'clean-contact')
cc_maxsep_conf_path = os.path.join(cc_path, 'med4_uniprotkb_proteome_UP000001026_2024_01_13_only_ids_in_headers_maxsep_confidence.csv')
cc_maxsep_path = os.path.join(cc_path, 'med4_uniprotkb_proteome_UP000001026_2024_01_13_only_ids_in_headers_maxsep.csv')
cc_pvalue_conf_path = os.path.join(cc_path, 'med4_uniprotkb_proteome_UP000001026_2024_01_13_only_ids_in_headers_pvalue_confidence.csv')
cc_pvalue_path = os.path.join(cc_path, 'med4_uniprotkb_proteome_UP000001026_2024_01_13_only_ids_in_headers_pvalue.csv')

# clean results
clean_path = os.path.join('..', 'raw-data', 'clean')
clean_maxsep_conf_path = os.path.join(clean_path, 'med4_uniprotkb_proteome_UP000001026_2024_01_13_maxsep.csv')
clean_pvalue_path = os.path.join(clean_path, 'med4_uniprotkb_proteome_UP000001026_2024_01_13_pvalue.csv')

# med4 annotations from BioCyc: https://cyanocyc.org/group?id=biocyc18-15682-3934545975
biocyc_path = os.path.join('..', 'raw-data', 'biocyc', 'Genes-of-a-reaction-from-All-instances-of-Reactions-in-Prochlorococcus-marinus-pastoris-CCMP1986.txt')

# read in original med4 fasta to get the header line annotations
med4_fasta_path = os.path.join('..', '..', 'computation', 'clean-contact', 'inputs', 'med4_uniprotkb_proteome_UP000001026_2024_01_13.fasta')

In [3]:
def process_csv_row(row):
    ''' 
    Helper function to read in csv data in correct format.
    '''
    # split the row by commas
    values = row.split(',')
    
    # first value goes to the first column
    first_value = values[0]
    
    # remaining values go to the second column as a list
    remaining_values = values[1:]
    
    return [first_value, remaining_values]

def process_results(data_path, my_cols):
    '''
    Read in csv results and give column names.
    '''
    # assumes no header lines in this input csv
    processed_data = []
    with open(data_path, 'r') as csv_file:
        for line in csv_file:
            processed_data.append(process_csv_row(line.strip()))
    processed_df = pd.DataFrame(processed_data, columns=my_cols)
    return processed_df

def remove_extra_id_format(s):
    '''
    From 'tr|Q7TU21|Q7TU21_PROMP' to 'Q7TU21'.
    '''
    parts = s.split('|')
    return f'{parts[1]}'

def format_ec(ec_number):
    ''' 
    Edit the BioCyc formatting for the EC number to match the format used by CLEAN and CLEAN-Contact.
    Instead of 'EC-1.2.1.38 // EC-1.2.1.106', use 'EC:1.2.1.38, EC:1.2.1.106'.
    '''
    if pd.isna(ec_number):
        return ec_number
    ec_number = ec_number.replace(' // ', ', ')
    entries = ec_number.split(', ') 
    clean_entries = [entry.replace('EC-', 'EC:') for entry in entries]
    return ', '.join(clean_entries)

def extract_ec_values(s):
    ''' 
    From the full format of the CLEAN or CLEAN-Contact results column to just the EC numbers. 
    before: '[EC:#.#.#.#/#_#]'
    after: 'EC:#.#.#.#'
    '''
    # regular expression handles n values in the last position of the ec number
    ec_values = re.findall(r'EC:\d+\.\d+\.\d+(?:\.\d+|\.n\d*)?', str(s))
    # return results as comma separated string, not actual list datatype
    return ', '.join(ec_values) 

def parse_fasta_ids(fasta_file):
    '''
    Parses a fasta file with UniProt ids and returns a dictionary of annotations and a dictionary with protein sequences 
    both with their corresponding UniProt ids.
    '''
    id_to_annotation = {}
    id_to_sequence = {}
    
    for record in SeqIO.parse(fasta_file, "fasta"):
        header = record.description
        id_part = header.split('|')[1]
        annotation_part = ' '.join(header.split(' ')[1:])
        sequence = str(record.seq)
        
        id_to_annotation[id_part] = annotation_part
        id_to_sequence[id_part] = sequence
        
    return id_to_annotation, id_to_sequence

def update_dataframe_with_annotations(fasta_file, dataframe):
    '''
    Add annotation info and protein sequences from fasta file to results df based on common UniProt ids.
    '''
    id_to_annotation, id_to_sequence = parse_fasta_ids(fasta_file)
    dataframe['Annotation'] = dataframe['UniProt'].map(id_to_annotation)
    dataframe['Sequence'] = dataframe['UniProt'].map(id_to_sequence)
    
    return dataframe

def extract_conf_no_std(string):
    '''
    Extract the confidence score part of the results, before the underscore.
    '''
    match = re.search(r'/([\d.]+)_', string)
    if match:
        return match.group(1)
    return None

def extract_conf_only(lst):
    '''
    From the full format of CLEAN-Contact results column to just the confidence score, without the standard deviation range.
    '''
    return ', '.join(filter(None, [extract_conf_no_std(s) for s in lst]))


def extract_std_range(string):
    '''
    Grab everything after the underscore.
    '''
    match = re.search(r'_([\d.]+)', string)
    if match:
        return match.group(1)
    return None

def extract_std_only(lst):
    '''
    Apply the extraction of standard deviation range of the confidence scores on lists of strings.
    '''
    return ', '.join(filter(None, [extract_std_range(s) for s in lst]))


def extract_value(string):
    '''
    Grab the value after the EC number prediction, so anything after /.
    '''
    match = re.search(r'/([\d.]+)', string)
    if match:
        return match.group(1)
    return None


def extract_from_list(lst):
    '''
    Apply the extraction of values after the EC number on lists of strings.
    '''
    return ', '.join(filter(None, [extract_value(s) for s in lst]))


## CLEAN-Contact 
When running CLEAN-Contact using either inference method with the GMM, the GMM confidence score results are in addition to the results without running the GMM. For the case study, each inference method was run once with the GMM. This is why we start with 2 datasets for maxsep and 2 datasets for pvalue; each inference method has results with and without the GMM confidence scores. Also note that CLEAN-Contact's GMM results include the confidence score as well as the standard deviation for the confidence score.


Maxsep
* results from using the maxsep inference method, with the GMM to get confidence scores
* results from using the maxsep inference method, without the GMM
* combine the maxsep results and share in this directory as **cc_maxsep_results_full.csv**
    * 1939 rows of data
    * 7 columns
        * 'UniProt': UniProt id
        * 'Annotation': header annotations from the fasta file
        * 'Sequence': protein sequences from the fasta file
        * 'CC maxsep EC': CLEAN-Contact's EC predictions from using the maxsep inference method
        * 'CC maxsep conf': CLEAN-Contact's confidence scores from using the maxsep inference method with the GMM
        * 'CC maxsep conf SD': CLEAN-Contact's confidence score standard deviation from using the maxsep inference method with GMM
        * 'CC maxsep cluster dist': CLEAN-Contact's EC prediction's distance from the cluster center from using the maxsep inference method

Pvalue
* results from using the pvalue inference method, with the GMM to get confidence scores
* results from using the pvalue inference methods, without the GMM
* combine the pvalue results and share in this directory as **cc_pvalue_results_full.csv**
    * 1939 rows of data
    * 7 columns
        * 'UniProt': UniProt id
        * 'Annotation': header annotations from the fasta file
        * 'Sequence': protein sequences from the fasta file
        * 'CC pvalue EC': CLEAN-Contact's EC predictions from using the pvalue inference method
        * 'CC pvalue conf': CLEAN-Contact's confidence scores from using the pvalue inference method with the GMM
        * 'CC pvalue conf SD': CLEAN-Contact's confidence score standard deviation from using the pvalue inference method with GMM
        * 'CC pvalue cluster dist': CLEAN-Contact's EC prediction's distance from the cluster center from using the pvalue inference method

In [4]:
# CLEAN-Contact maxsep results, with confidence scores and SD from using the gmm
cc_maxsep_conf = process_results(cc_maxsep_conf_path, ['UniProt', 'CC maxsep conf full'])
cc_maxsep_conf = update_dataframe_with_annotations(med4_fasta_path, cc_maxsep_conf)
cc_maxsep_conf['CC maxsep conf EC'] = cc_maxsep_conf['CC maxsep conf full'].apply(extract_ec_values)
cc_maxsep_conf['CC maxsep conf'] = cc_maxsep_conf['CC maxsep conf full'].apply(extract_conf_only)
cc_maxsep_conf['CC maxsep conf SD'] = cc_maxsep_conf['CC maxsep conf full'].apply(extract_std_only)
cc_maxsep_conf

,UniProt,CC maxsep conf full,Annotation,Sequence,CC maxsep conf EC,CC maxsep conf,CC maxsep conf SD
0,Q7TU21,[EC:1.2.1.11/0.9863_0.0041],Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,0.9863,0.0041
1,Q7UZH7,[EC:6.3.4.2/0.9961_0.0017],CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,0.9961,0.0017
2,Q7UZR6,[EC:6.3.2.1/0.8120_0.0551],Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:6.3.2.1,0.8120,0.0551
3,Q7V010,[EC:2.1.1.192/0.9923_0.0025],Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,EC:2.1.1.192,0.9923,0.0025
4,Q7V0C4,[EC:2.7.11.1/0.0911_0.0452],Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,0.0911,0.0452
...,...,...,...,...,...,...,...
1934,Q7V3R0,[EC:4.2.3.187/0.0014_0.0012],Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:4.2.3.187,0.0014,0.0012
1935,Q7V3R1,[EC:2.7.1.66/0.0014_0.0012],Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:2.7.1.66,0.0014,0.0012
1936,Q7V3R2,[EC:1.17.99.6/0.9914_0.0028],4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,EC:1.17.99.6,0.9914,0.0028
1937,Q7V3R3,[EC:3.5.2.6/0.0787_0.0403],Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,EC:3.5.2.6,0.0787,0.0403


In [5]:
# CLEAN-Contact maxsep results, with distance from cluster center
cc_maxsep = process_results(cc_maxsep_path, ['UniProt', 'CC maxsep full'])
cc_maxsep['CC maxsep EC'] = cc_maxsep['CC maxsep full'].apply(extract_ec_values)
cc_maxsep['CC maxsep cluster dist'] = cc_maxsep['CC maxsep full'].apply(extract_from_list)
cc_maxsep

,UniProt,CC maxsep full,CC maxsep EC,CC maxsep cluster dist
0,Q7TU21,[EC:1.2.1.11/4.3332],EC:1.2.1.11,4.3332
1,Q7UZH7,[EC:6.3.4.2/2.8527],EC:6.3.4.2,2.8527
2,Q7UZR6,[EC:6.3.2.1/5.8351],EC:6.3.2.1,5.8351
3,Q7V010,[EC:2.1.1.192/3.8433],EC:2.1.1.192,3.8433
4,Q7V0C4,[EC:2.7.11.1/7.2259],EC:2.7.11.1,7.2259
...,...,...,...,...
1934,Q7V3R0,[EC:4.2.3.187/8.4480],EC:4.2.3.187,8.4480
1935,Q7V3R1,[EC:2.7.1.66/8.4444],EC:2.7.1.66,8.4444
1936,Q7V3R2,[EC:1.17.99.6/3.9525],EC:1.17.99.6,3.9525
1937,Q7V3R3,[EC:3.5.2.6/7.2768],EC:3.5.2.6,7.2768


In [6]:
# combine and format the maxsep results if they have the same UniProt id and EC predictions
if cc_maxsep_conf['CC maxsep conf EC'].equals(cc_maxsep['CC maxsep EC']):
    print("The columns match. Proceeding to combine the DataFrames.")
    
    # Combine the DataFrames on 'UniProt' and maxsep columns
    cc_maxsep_results_full = pd.merge(cc_maxsep_conf, cc_maxsep, left_on=['UniProt', 'CC maxsep conf EC'], right_on=['UniProt', 'CC maxsep EC'])
    
    # Drop the redundant maxsep ec column 
    cc_maxsep_results_full.drop(columns=['CC maxsep conf EC'], inplace=True)
    
else:
    print("The columns do not match.")

cc_maxsep_results_full.drop(columns=['CC maxsep conf full', 'CC maxsep full'], inplace=True)
col_to_move = cc_maxsep_results_full.pop('CC maxsep EC')
cc_maxsep_results_full.insert(3, 'CC maxsep EC', col_to_move)

The columns match. Proceeding to combine the DataFrames.


In [7]:
# cc_maxsep_results_full.to_csv('cc_maxsep_results_full.csv', index=False)
cc_maxsep_results_full

,UniProt,Annotation,Sequence,CC maxsep EC,CC maxsep conf,CC maxsep conf SD,CC maxsep cluster dist
0,Q7TU21,Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,0.9863,0.0041,4.3332
1,Q7UZH7,CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,0.9961,0.0017,2.8527
2,Q7UZR6,Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:6.3.2.1,0.8120,0.0551,5.8351
3,Q7V010,Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,EC:2.1.1.192,0.9923,0.0025,3.8433
4,Q7V0C4,Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,0.0911,0.0452,7.2259
...,...,...,...,...,...,...,...
1934,Q7V3R0,Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:4.2.3.187,0.0014,0.0012,8.4480
1935,Q7V3R1,Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:2.7.1.66,0.0014,0.0012,8.4444
1936,Q7V3R2,4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,EC:1.17.99.6,0.9914,0.0028,3.9525
1937,Q7V3R3,Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,EC:3.5.2.6,0.0787,0.0403,7.2768


In [8]:
# CLEAN-Contact pvalue results, with confidence scores and SD from using the gmm
cc_pvalue_conf = process_results(cc_pvalue_conf_path, ['UniProt', 'CC pvalue conf full'])
cc_pvalue_conf = update_dataframe_with_annotations(med4_fasta_path, cc_pvalue_conf)
cc_pvalue_conf['CC pvalue conf EC'] = cc_pvalue_conf['CC pvalue conf full'].apply(extract_ec_values)
cc_pvalue_conf['CC pvalue conf'] = cc_pvalue_conf['CC pvalue conf full'].apply(extract_conf_only)
cc_pvalue_conf['CC pvalue conf SD'] = cc_pvalue_conf['CC pvalue conf full'].apply(extract_std_only)
cc_pvalue_conf

,UniProt,CC pvalue conf full,Annotation,Sequence,CC pvalue conf EC,CC pvalue conf,CC pvalue conf SD
0,Q7TU21,[EC:1.2.1.11/0.9863_0.0041],Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,0.9863,0.0041
1,Q7UZH7,[EC:6.3.4.2/0.9961_0.0017],CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,0.9961,0.0017
2,Q7UZR6,[EC:6.3.2.1/0.8120_0.0551],Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:6.3.2.1,0.8120,0.0551
3,Q7V010,[EC:2.1.1.192/0.9923_0.0025],Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,EC:2.1.1.192,0.9923,0.0025
4,Q7V0C4,[EC:2.7.11.1/0.0911_0.0452],Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,0.0911,0.0452
...,...,...,...,...,...,...,...
1934,Q7V3R0,[EC:4.2.3.187/0.0014_0.0012],Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:4.2.3.187,0.0014,0.0012
1935,Q7V3R1,[EC:2.7.1.66/0.0014_0.0012],Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:2.7.1.66,0.0014,0.0012
1936,Q7V3R2,"[EC:1.17.99.6/0.9914_0.0028, EC:4.1.99.3/0.000...",4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,"EC:1.17.99.6, EC:4.1.99.3, EC:4.1.99.13","0.9914, 0.0000, 0.0000","0.0028, 0.0000, 0.0000"
1937,Q7V3R3,"[EC:3.5.2.6/0.0787_0.0403, EC:3.1.1.107/0.0034...",Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,"EC:3.5.2.6, EC:3.1.1.107","0.0787, 0.0034","0.0403, 0.0027"


In [9]:
# CLEAN-Contact pvalue results, with distance from cluster center
cc_pvalue = process_results(cc_pvalue_path, ['UniProt', 'CC pvalue full'])
cc_pvalue['CC pvalue EC'] = cc_pvalue['CC pvalue full'].apply(extract_ec_values)
cc_pvalue['CC pvalue cluster dist'] = cc_pvalue['CC pvalue full'].apply(extract_from_list)
cc_pvalue

,UniProt,CC pvalue full,CC pvalue EC,CC pvalue cluster dist
0,Q7TU21,[EC:1.2.1.11/4.3332],EC:1.2.1.11,4.3332
1,Q7UZH7,[EC:6.3.4.2/2.8527],EC:6.3.4.2,2.8527
2,Q7UZR6,[EC:6.3.2.1/5.8351],EC:6.3.2.1,5.8351
3,Q7V010,[EC:2.1.1.192/3.8433],EC:2.1.1.192,3.8433
4,Q7V0C4,[EC:2.7.11.1/7.2259],EC:2.7.11.1,7.2259
...,...,...,...,...
1934,Q7V3R0,[EC:4.2.3.187/8.4480],EC:4.2.3.187,8.4480
1935,Q7V3R1,[EC:2.7.1.66/8.4444],EC:2.7.1.66,8.4444
1936,Q7V3R2,"[EC:1.17.99.6/3.9525, EC:4.1.99.3/9.3495, EC:4...","EC:1.17.99.6, EC:4.1.99.3, EC:4.1.99.13","3.9525, 9.3495, 9.8844"
1937,Q7V3R3,"[EC:3.5.2.6/7.2768, EC:3.1.1.107/8.2078]","EC:3.5.2.6, EC:3.1.1.107","7.2768, 8.2078"


In [10]:
# combine and format the pvalue results if they have the same UniProt id and EC predictions
if cc_pvalue_conf['CC pvalue conf EC'].equals(cc_pvalue['CC pvalue EC']):
    print("The columns match. Proceeding to combine the DataFrames.")
    
    # Combine the DataFrames on 'UniProt' and pvalue columns
    cc_pvalue_results_full = pd.merge(cc_pvalue_conf, cc_pvalue, left_on=['UniProt', 'CC pvalue conf EC'], right_on=['UniProt', 'CC pvalue EC'])
    
    # Drop the redundant pvalue ec column 
    cc_pvalue_results_full.drop(columns=['CC pvalue conf EC'], inplace=True)
    
else:
    print("The columns do not match.")

cc_pvalue_results_full.drop(columns=['CC pvalue conf full', 'CC pvalue full'], inplace=True)
col_to_move = cc_pvalue_results_full.pop('CC pvalue EC')
cc_pvalue_results_full.insert(3, 'CC pvalue EC', col_to_move)

The columns match. Proceeding to combine the DataFrames.


In [11]:
#cc_pvalue_results_full.to_csv('cc_pvalue_results_full.csv', index=False)
cc_pvalue_results_full

,UniProt,Annotation,Sequence,CC pvalue EC,CC pvalue conf,CC pvalue conf SD,CC pvalue cluster dist
0,Q7TU21,Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,0.9863,0.0041,4.3332
1,Q7UZH7,CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,0.9961,0.0017,2.8527
2,Q7UZR6,Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:6.3.2.1,0.8120,0.0551,5.8351
3,Q7V010,Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,EC:2.1.1.192,0.9923,0.0025,3.8433
4,Q7V0C4,Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,0.0911,0.0452,7.2259
...,...,...,...,...,...,...,...
1934,Q7V3R0,Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:4.2.3.187,0.0014,0.0012,8.4480
1935,Q7V3R1,Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:2.7.1.66,0.0014,0.0012,8.4444
1936,Q7V3R2,4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,"EC:1.17.99.6, EC:4.1.99.3, EC:4.1.99.13","0.9914, 0.0000, 0.0000","0.0028, 0.0000, 0.0000","3.9525, 9.3495, 9.8844"
1937,Q7V3R3,Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,"EC:3.5.2.6, EC:3.1.1.107","0.0787, 0.0034","0.0403, 0.0027","7.2768, 8.2078"


## CLEAN
CLEAN only provides the GMM option for the maxsep inference method, not pvalue, and those results are not in addition to the maxsep method results without the GMM. For the case study each inference method was run once, with the GMM if possible. This is why the CLEAN results start with 1 dataset for maxsep (only with GMM) and 1 dataset for pvalue (no GMM option available). Also note that CLEAN's GMM results only include the confidence score; no standard deviation for the confidence score.

Maxsep with GMM
* results from using the maxsep inference method, with the GMM to get confidence scores

Pvalue
* results from using the pvalue inference method, without the GMM since there's not a GMM option available 

Combined
* combine the maxsep and pvalue results and share in this directory as **clean_combined_results_full.csv**
    * 1942 rows of data
    * 7 columns
        * 'UniProt': UniProt id
        * 'Annotation': header annotations from the fasta file
        * 'Sequence': protein sequences from the fasta file
        * 'Clean maxsep EC': CLEAN's EC predictions from using the maxsep inference method
        * 'Clean maxsep conf': CLEAN's confidence scores from using the maxsep inference method with the GMM
        * 'Clean pvalue EC': CLEAN's EC predictions from using the pvalue inference method
        * 'Clean pvalue cluster dist': CLEAN's EC prediction's distance from the cluster center from using the pvalue inference method

In [12]:
# CLEAN's maxsep results, with confidence scores from using the gmm
clean_maxsep_conf = process_results(clean_maxsep_conf_path, ['UniProt', 'Clean maxsep conf full'])
clean_maxsep_conf['UniProt'] = clean_maxsep_conf['UniProt'].apply(remove_extra_id_format)
clean_maxsep_conf = update_dataframe_with_annotations(med4_fasta_path, clean_maxsep_conf)
clean_maxsep_conf['Clean maxsep EC'] = clean_maxsep_conf['Clean maxsep conf full'].apply(extract_ec_values)
clean_maxsep_conf['Clean maxsep conf'] = clean_maxsep_conf['Clean maxsep conf full'].apply(extract_from_list)
clean_maxsep_conf

,UniProt,Clean maxsep conf full,Annotation,Sequence,Clean maxsep EC,Clean maxsep conf
0,Q7TU21,[EC:1.2.1.11/0.9935],Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,0.9935
1,Q7UZH7,[EC:6.3.4.2/0.9976],CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,0.9976
2,Q7UZR6,[EC:2.7.4.25/0.2534],Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:2.7.4.25,0.2534
3,Q7V010,[EC:2.1.1.192/0.9917],Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,EC:2.1.1.192,0.9917
4,Q7V0C4,[EC:2.7.11.1/0.0451],Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,0.0451
...,...,...,...,...,...,...
1937,Q7V3R0,[EC:2.5.1.17/0.0002],Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:2.5.1.17,0.0002
1938,Q7V3R1,[EC:1.1.5.2/0.0001],Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:1.1.5.2,0.0001
1939,Q7V3R2,[EC:1.17.99.6/0.9886],4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,EC:1.17.99.6,0.9886
1940,Q7V3R3,[EC:5.2.1.8/0.0077],Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,EC:5.2.1.8,0.0077


In [13]:
# CLEAN's pvalue results, with distance from cluster center
clean_pvalue = process_results(clean_pvalue_path, ['UniProt', 'Clean pvalue full'])
clean_pvalue['UniProt'] = clean_pvalue['UniProt'].apply(remove_extra_id_format)
clean_pvalue = update_dataframe_with_annotations(med4_fasta_path, clean_pvalue)
clean_pvalue['Clean pvalue EC'] = clean_pvalue['Clean pvalue full'].apply(extract_ec_values)
clean_pvalue['Clean pvalue cluster dist'] = clean_pvalue['Clean pvalue full'].apply(extract_from_list)
clean_pvalue

,UniProt,Clean pvalue full,Annotation,Sequence,Clean pvalue EC,Clean pvalue cluster dist
0,Q7TU21,[EC:1.2.1.11/3.7585],Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,3.7585
1,Q7UZH7,[EC:6.3.4.2/2.9574],CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,2.9574
2,Q7UZR6,[EC:2.7.4.25/6.4352],Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:2.7.4.25,6.4352
3,Q7V010,"[EC:2.1.1.192/3.9242, EC:2.1.1.224/4.9630]",Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,"EC:2.1.1.192, EC:2.1.1.224","3.9242, 4.9630"
4,Q7V0C4,[EC:2.7.11.1/7.0824],Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,7.0824
...,...,...,...,...,...,...
1937,Q7V3R0,[EC:2.5.1.17/8.6199],Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:2.5.1.17,8.6199
1938,Q7V3R1,[EC:1.1.5.2/8.6942],Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:1.1.5.2,8.6942
1939,Q7V3R2,[EC:1.17.99.6/4.1148],4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,EC:1.17.99.6,4.1148
1940,Q7V3R3,[EC:5.2.1.8/7.6222],Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,EC:5.2.1.8,7.6222


In [14]:
# combine clean results from maxsep and pvalue
clean_combined_results_full = pd.merge(clean_maxsep_conf, clean_pvalue, on=['UniProt', 'Annotation', 'Sequence'])
clean_combined_results_full.drop(columns=['Clean maxsep conf full', 'Clean pvalue full'], inplace=True)
#clean_combined_results_full.to_csv('clean_combined_results_full.csv', index=False)
clean_combined_results_full  

,UniProt,Annotation,Sequence,Clean maxsep EC,Clean maxsep conf,Clean pvalue EC,Clean pvalue cluster dist
0,Q7TU21,Aspartate-semialdehyde dehydrogenase OS=Prochl...,MRNSPFLPNRPLKVAVLGSSGAVGSELLKILEERDFPISELVLLSS...,EC:1.2.1.11,0.9935,EC:1.2.1.11,3.7585
1,Q7UZH7,CTP synthase OS=Prochlorococcus marinus subsp....,MSKFVFVTGGVVSSIGKGIVAASLGRLLKSRGYSVSILKLDPYLNV...,EC:6.3.4.2,0.9976,EC:6.3.4.2,2.9574
2,Q7UZR6,Bifunctional pantoate ligase/cytidylate kinase...,MNKIIIRKTEDLKEWRRNLKCDINFIPTMGNLHDGHQKLISTAQSS...,EC:2.7.4.25,0.2534,EC:2.7.4.25,6.4352
3,Q7V010,Probable dual-specificity RNA methyltransferas...,MKNLLGCSVKDLEKIALNYGQAAFRGRQIYNWLYNYKNRSKSIDEI...,EC:2.1.1.192,0.9917,"EC:2.1.1.192, EC:2.1.1.224","3.9242, 4.9630"
4,Q7V0C4,Circadian clock oscillator protein KaiC OS=Pro...,MKDKKISKSIKMQVQKIPTGIEGFDDVCRGGLPAARSTLVSGTSGT...,EC:2.7.11.1,0.0451,EC:2.7.11.1,7.0824
...,...,...,...,...,...,...,...
1937,Q7V3R0,Antitermination protein NusB OS=Prochlorococcu...,MMHNRSLSRELSLLSLGLIKDTADLELNKIQIDEIFESALDSLINH...,EC:2.5.1.17,0.0002,EC:2.5.1.17,8.6199
1938,Q7V3R1,Transporter OS=Prochlorococcus marinus subsp. ...,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...,EC:1.1.5.2,0.0001,EC:1.1.5.2,8.6942
1939,Q7V3R2,4Fe-4S ferredoxin-type domain-containing prote...,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...,EC:1.17.99.6,0.9886,EC:1.17.99.6,4.1148
1940,Q7V3R3,Uncharacterized protein OS=Prochlorococcus mar...,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...,EC:5.2.1.8,0.0077,EC:5.2.1.8,7.6222


## BioCyc
BioCyc data on Prochlorococcus marinus subsp. pastoris str. CCMP1986, also known as MED4.

Found and fixed an error where the UniProt id 'Q7V3L5' was incorrectly duplicated. One entry was correct, but the other was supposed to be 'Q7V2M2'. This was updated here in our data as well as reported to BioCyc, where the data has been updated in their records. This processing step also updated the EC number column with the EC number formatting to match CLEAN and CLEAN-Contact's EC number formatting.

Update and share in this directory as **biocyc_updated.csv**.
   * 728 rows of data
   * 7 columns
       * 'UniProt': UniProt id
       * 'Genes of a reaction from All instances of Reactions in Prochlorococcus marinus pastoris CCMP1986': genes involved in these reactions
       * 'Matches': reaction matches
       * 'Accession-1': accession number from another database
       * 'Accession-2': accession number from another database
       * 'Gene products (polypeptides and RNAs only)': the gene products encoded by the genes in these reactions
       * 'BioCyc EC': BioCyc's EC number, formatted to match CLEAN and CLEAN-Contact
       

In [15]:
biocyc = pd.read_csv(biocyc_path, delimiter='\t')
biocyc['UniProt'].value_counts()

UniProt
Q7V3L5    2
Q7V2E7    1
Q7TU77    1
Q7V189    1
Q7V177    1
         ..
Q7V2Q1    1
Q7V2G7    1
Q7V278    1
Q7V121    1
Q7V116    1
Name: count, Length: 720, dtype: int64

In [16]:
# correct the duplicated UniProt id
biocyc_updated = biocyc.copy()
biocyc_updated.loc[(biocyc_updated['UniProt'] == 'Q7V3L5') & (biocyc_updated['Genes of a reaction from All instances of Reactions in Prochlorococcus marinus pastoris CCMP1986'] == 'fabG'), 'UniProt'] = 'Q7V2M2'

# view these changes
biocyc_updated[biocyc_updated['UniProt'].isin(['Q7V3L5','Q7V2M2'])]

,UniProt,Genes of a reaction from All instances of Reactions in Prochlorococcus marinus pastoris CCMP1986,Matches,EC-Number,Accession-1,Accession-2,Gene products (polypeptides and RNAs only)
3,Q7V3L5,accC,a [biotin carboxyl-carrier-protein dimer]-N6-b...,EC-6.3.4.14,PMM0060,TX50_RS00320,acetyl-CoA carboxylase biotin carboxylase subunit
140,Q7V2M2,fabG,3-oxo-meristoyl-[acyl-carrier protein] reducta...,EC-1.1.1.100,PMM0453,TX50_RS02440,3-oxoacyl-[acyl-carrier-protein] reductase


In [17]:
# update the EC number format to match the format in CLEAN and CLEAN-Contact's results
biocyc_updated['BioCyc EC'] = biocyc_updated['EC-Number'].apply(format_ec)
biocyc_updated.drop(columns=['EC-Number'], inplace=True)
#biocyc_updated.to_csv('biocyc_updated.csv', index=False)
biocyc_updated

,UniProt,Genes of a reaction from All instances of Reactions in Prochlorococcus marinus pastoris CCMP1986,Matches,Accession-1,Accession-2,Gene products (polypeptides and RNAs only),BioCyc EC
0,Q7V2E7,aarA,octadecanal + acyl-carrier protein + NAD(P)+ ...,PMM0533,TX50_RS02855,long-chain acyl-[acyl-carrier-protein] reductase,EC:1.2.1.80
1,Q7V2E6,accA,acetyl-CoA + a [carboxyl-carrier protein dimer...,PMM0534,TX50_RS02860,acetyl-CoA carboxylase carboxyltransferase sub...,EC:2.1.3.15
2,Q7TUH9,accB,ATP + acetyl-CoA + hydrogencarbonate -> ADP ...,PMM0027,TX50_RS00150,acetyl-CoA carboxylase biotin carboxyl carrier...,EC:6.4.1.2
3,Q7V3L5,accC,a [biotin carboxyl-carrier-protein dimer]-N6-b...,PMM0060,TX50_RS00320,acetyl-CoA carboxylase biotin carboxylase subunit,EC:6.3.4.14
4,Q7V1S1,accD,acetyl-CoA + a [carboxyl-carrier protein dimer...,PMM0784,TX50_RS04185,acetyl-CoA carboxylase carboxyltransferase sub...,EC:2.1.3.15
...,...,...,...,...,...,...,...
723,NaN,ycf12,RXN-15490 // oxygen[in] + 4 H+[in] + 4 e-[memb...,NaN,NaN,photosystem II reaction center protein Ycf12,"EC:1.1.3, EC:1.10.3.9"
724,Q7V0W4,ychF,ATP + H2O -> ADP + phosphate + H+,PMM1138,TX50_RS06135,redox-regulated ATPase YchF,EC:3.6.1.15
725,Q7V3G0,zds,beta-zeacarotene + an electron-transfer quinon...,PMM0115,TX50_RS00605,"9,9'-di-cis-zeta-carotene desaturase",EC:1.3.5.6
726,Q7V3E5,zntA,a metal cation[out] + ATP + H2O -> a metal c...,PMM0131,TX50_RS00690,P-type cation transporter,NaN
